In [1]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
writer = SummaryWriter()

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [31]:
t = torch.tensor([1.0,2000.0,3.0])
noise = torch.randn_like(t)
noise.multiply(1)

tensor([-0.6847, -0.0220, -0.2742])

In [6]:
VALIDATION_SIZE = 1000
df = pd.read_csv("/home/vaclavmatejka/devel/janestreet/jane-street-market-prediction/train.csv", nrows=10000)


In [7]:
df = df.fillna(0)
df.loc[df['resp'] <= 0, 'trade'] = 0
df.loc[df['resp'] > 0, 'trade'] = 1

In [8]:
df = df.sample(frac=1).reset_index()
df

,index,date,weight,resp_1,resp_2,resp_3,resp_4,resp,feature_0,feature_1,...,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,ts_id,trade
0,4229,0,3.132084,-0.000383,-0.001703,-0.008157,-0.015278,-0.006225,1,-0.992881,...,-2.933236,-2.800955,-1.663217,-2.655227,-3.242365,-2.211206,-2.933141,-2.070658,4229,0.0
1,8629,1,0.000000,0.001986,0.001278,-0.011261,-0.028610,-0.018922,-1,0.350301,...,1.499540,0.450863,1.331610,0.842357,1.915589,0.559628,1.769559,0.507244,8629,0.0
2,883,0,0.097277,-0.004123,-0.008668,-0.016509,-0.036601,-0.030446,-1,0.092131,...,0.485573,-0.850156,0.978036,0.419343,1.406676,0.220234,1.120499,-0.016153,883,0.0
3,6793,1,0.407724,0.012262,0.025224,0.036325,0.013801,0.015037,-1,0.723173,...,6.092348,3.242825,5.299754,3.832930,7.550844,3.465466,6.729721,3.098703,6793,1.0
4,185,0,8.814972,0.000033,-0.000701,-0.001441,-0.003996,-0.003313,-1,-0.805544,...,0.212045,3.158191,0.455739,5.246220,0.700746,4.316125,0.426289,3.475867,185,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1353,0,5.253691,-0.001766,-0.005905,-0.012264,-0.032458,-0.031542,1,5.403006,...,0.914035,-0.312832,0.915264,-0.098789,0.642980,-1.074432,0.540627,-1.023037,1353,0.0
9996,9051,1,0.113178,-0.000742,-0.006647,-0.006370,0.001137,0.001151,1,0.956896,...,0.528069,-0.702367,0.562997,-0.521322,0.648534,-0.689799,0.521436,-0.775917,9051,1.0
9997,1474,0,2.024854,-0.002039,0.004874,0.019715,0.026534,0.016833,1,2.754956,...,0.912643,-0.865258,1.204346,-0.103448,1.804606,-0.066528,1.254176,-0.484706,1474,1.0
9998,9140,1,3.860834,-0.002775,-0.003696,-0.003341,-0.010295,-0.008862,1,2.125217,...,0.231419,0.821957,-0.119893,0.295155,-0.099360,0.284603,0.099490,0.524413,9140,0.0


In [6]:
class JaneStreetDataset(Dataset):

    # Constructor with defult values
    def __init__(self, df, transform=None):
        self.df = df
        self.len = len(self.df)
        self.transform = transform

    # Getter
    def __getitem__(self, index):
        sample = torch.from_numpy(self.df.iloc[index]["feature_0":"feature_129"].values), torch.tensor(self.df.iloc[index]["resp"])
        sample = sample[0].to(device), sample[1].to(device)
        if self.transform:
            sample = self.transform(sample)
        return sample

    # Get Length
    def __len__(self):
        return self.len

In [7]:
df.iloc[2]["resp"]

0.0239701262920541

In [8]:
df.iloc[2]["feature_0":"feature_129"].values

array([-1.        ,  0.81278043, -0.25615584,  0.80646267,  0.4002206 ,
       -0.61418779, -0.35480005,  0.        ,  0.        ,  5.44826084,
        2.66802875,  0.        ,  0.        ,  3.83634173,  2.18325752,
        3.90269776,  3.04543074,  0.        ,  0.        , -1.1410816 ,
       -0.97996232,  0.        ,  0.        , -1.15758467, -0.96680317,
       -1.43097261, -1.10343163,  0.        ,  0.        ,  5.13155872,
        4.31471422,  0.        ,  0.        ,  4.2263413 ,  3.17364035,
        5.99151344,  4.14229764, -0.16792715, -0.12477773,  0.74932579,
        0.71582445, -0.03900658,  0.18632062,  2.32388738,  0.16226144,
        0.23798736, -0.35022138, -0.13803296, -0.51628111, -0.7035434 ,
       -0.55695406, -0.81690971, -0.45584133,  2.3832265 ,  3.47441639,
        0.        ,  0.88324663, -0.08424736,  0.62256055,  0.95361872,
       -1.12877427, -0.94650748, -1.47076236, -1.59458685, -4.34619924,
       -2.27667794, -1.41765152, -2.16636207, -2.07710993,  0.69

In [9]:
DROPOUT_P = 0.0
big_number = 1000
small_number = 500
model = torch.nn.Sequential(
    torch.nn.Linear(130, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, big_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(big_number),
    torch.nn.Linear(big_number, small_number),
    nn.Dropout(p=DROPOUT_P),
    torch.nn.ReLU(),
    nn.BatchNorm1d(small_number),
    torch.nn.Linear(small_number, 1),
)
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)


/home/vaclavmatejka/adata/anaconda3/envs/coursera/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [10]:
janestreet_train = JaneStreetDataset(df=df.iloc[:-VALIDATION_SIZE])
janestreet_validation = JaneStreetDataset(df=df.iloc[-VALIDATION_SIZE:])
BATCH_SIZE = 100
N_EPOCHS = 50
LR = 0.05
train_loader = torch.utils.data.DataLoader(dataset=janestreet_train, batch_size=BATCH_SIZE, drop_last=True)
validation_loader = torch.utils.data.DataLoader(dataset=janestreet_validation, batch_size=BATCH_SIZE, drop_last=True)

In [11]:
model = model.float()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.001)
criterion = nn.MSELoss()

In [12]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=None):
    accuracy_list = []
    loss_list = []
    for epoch in range(n_epochs):
        for x, y in train_loader:
            model.train()
            optimizer.zero_grad()
            z = model(x.float())
            y = y.view(-1, 1).float()
            loss = criterion(z, y)
            writer.add_scalar("Train data", loss, epoch)
            loss.backward()
            optimizer.step()
            loss_list.append(loss.data)
        # perform a prediction on the validation data
        for x_test, y_test in validation_loader:
            model.eval()
            z = model(x_test.float())
            y = y.view(-1, 1)
            loss = criterion(z, y.float())
            writer.add_scalar("Validation data", loss, epoch)
            accuracy_list.append(loss.data)
    return accuracy_list, loss_list

In [13]:
accuracy_list, loss_list = train_model(model, train_loader, validation_loader, optimizer, criterion, n_epochs=N_EPOCHS)

In [14]:
model.eval()

Sequential(
  (0): Linear(in_features=130, out_features=500, bias=True)
  (1): Dropout(p=0.0, inplace=False)
  (2): ReLU()
  (3): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): Linear(in_features=500, out_features=1000, bias=True)
  (5): Dropout(p=0.0, inplace=False)
  (6): ReLU()
  (7): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): Linear(in_features=1000, out_features=1000, bias=True)
  (9): Dropout(p=0.0, inplace=False)
  (10): ReLU()
  (11): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): Linear(in_features=1000, out_features=1000, bias=True)
  (13): Dropout(p=0.0, inplace=False)
  (14): ReLU()
  (15): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (16): Linear(in_features=1000, out_features=500, bias=True)
  (17): Dropout(p=0.0, inplace=False)
  (18): ReLU()
  (19): BatchNorm1d(500, eps=1e-05, momentum=0.1, affine=True, 

In [15]:
index=2000
#a = torch.from_numpy(df.iloc[index]["feature_0":"feature_129"].values)
a = df.iloc[index]["feature_0":"feature_129"].values
resp = torch.tensor(df.iloc[index]["resp"])

r = []
for one in range(BATCH_SIZE):
    r.append(a)
r = np.array(r)
model_input =torch.from_numpy(r).float().to(device)
z = model(model_input)

(resp, z[0])

(tensor(-0.0036), tensor([0.0013], device='cuda:0', grad_fn=<SelectBackward>))

In [16]:
accuracy_list

[tensor(0.0186, device='cuda:0'),
 tensor(0.0153, device='cuda:0'),
 tensor(0.0061, device='cuda:0'),
 tensor(0.0036, device='cuda:0'),
 tensor(0.0037, device='cuda:0'),
 tensor(0.0038, device='cuda:0'),
 tensor(0.0044, device='cuda:0'),
 tensor(0.0044, device='cuda:0'),
 tensor(0.0035, device='cuda:0'),
 tensor(0.0068, device='cuda:0'),
 tensor(0.0038, device='cuda:0'),
 tensor(0.0051, device='cuda:0'),
 tensor(0.0029, device='cuda:0'),
 tensor(0.0022, device='cuda:0'),
 tensor(0.0020, device='cuda:0'),
 tensor(0.0028, device='cuda:0'),
 tensor(0.0023, device='cuda:0'),
 tensor(0.0026, device='cuda:0'),
 tensor(0.0024, device='cuda:0'),
 tensor(0.0038, device='cuda:0'),
 tensor(0.0022, device='cuda:0'),
 tensor(0.0030, device='cuda:0'),
 tensor(0.0020, device='cuda:0'),
 tensor(0.0017, device='cuda:0'),
 tensor(0.0014, device='cuda:0'),
 tensor(0.0021, device='cuda:0'),
 tensor(0.0018, device='cuda:0'),
 tensor(0.0017, device='cuda:0'),
 tensor(0.0019, device='cuda:0'),
 tensor(0.0028

In [17]:
loss_list

[tensor(1.7777, device='cuda:0'),
 tensor(260.3978, device='cuda:0'),
 tensor(24.2435, device='cuda:0'),
 tensor(138.1919, device='cuda:0'),
 tensor(91.5306, device='cuda:0'),
 tensor(26.3294, device='cuda:0'),
 tensor(12.3440, device='cuda:0'),
 tensor(25.0281, device='cuda:0'),
 tensor(25.7195, device='cuda:0'),
 tensor(17.7737, device='cuda:0'),
 tensor(10.2398, device='cuda:0'),
 tensor(8.9544, device='cuda:0'),
 tensor(7.7700, device='cuda:0'),
 tensor(7.5105, device='cuda:0'),
 tensor(5.6331, device='cuda:0'),
 tensor(3.2646, device='cuda:0'),
 tensor(2.7660, device='cuda:0'),
 tensor(2.6691, device='cuda:0'),
 tensor(3.4571, device='cuda:0'),
 tensor(4.3666, device='cuda:0'),
 tensor(4.8957, device='cuda:0'),
 tensor(4.0818, device='cuda:0'),
 tensor(2.4752, device='cuda:0'),
 tensor(0.9717, device='cuda:0'),
 tensor(0.5463, device='cuda:0'),
 tensor(0.7192, device='cuda:0'),
 tensor(1.0505, device='cuda:0'),
 tensor(1.6187, device='cuda:0'),
 tensor(1.4869, device='cuda:0'),
 t